In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

# Загрузка данных
data = pd.read_csv('your_dataset.csv')  # Замените на имя файла

# Преобразование timestamp в datetime
data['departure_time'] = pd.to_datetime(data['departure_time'], unit='s')
data['arrival_time'] = pd.to_datetime(data['arrival_time'], unit='s')

# Извлечение временных характеристик
data['year'] = data['departure_time'].dt.year
data['month'] = data['departure_time'].dt.month
data['day'] = data['departure_time'].dt.day
data['hour'] = data['departure_time'].dt.hour

# Добавление новых характеристик
data['travel_time'] = (data['arrival_time'] - data['departure_time']).dt.total_seconds() / 3600  # Время в пути в часах
data['average_speed'] = data['road_length'] / data['travel_time']  # Средняя скорость

# Проверка данных
print(data.head())


In [ ]:
# Выбор значимых признаков для кластеризации
features = data[['road_length', 'travel_time', 'tips_amount']]

# Нормализация данных (опционально)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Кластеризация
kmeans = KMeans(n_clusters=3, random_state=42)
data['cluster'] = kmeans.fit_predict(features_scaled)

# Средние значения характеристик по кластерам
cluster_summary = data.groupby(['year', 'cluster']).mean()
print(cluster_summary)


In [ ]:
# Сравнение характеристик кластеров по годам
for year in data['year'].unique():
    print(f"Год {year}:")
    display(data[data['year'] == year].groupby('cluster').mean())


In [ ]:
# Динамика средней длины поездки по кластерам
avg_length = data.groupby(['year', 'cluster'])['road_length'].mean().reset_index()

sns.lineplot(data=avg_length, x='year', y='road_length', hue='cluster')
plt.title('Динамика длины поездок по кластерам')
plt.xlabel('Год')
plt.ylabel('Средняя длина поездки')
plt.show()

# Динамика средней скорости по кластерам
avg_speed = data.groupby(['year', 'cluster'])['average_speed'].mean().reset_index()

sns.lineplot(data=avg_speed, x='year', y='average_speed', hue='cluster')
plt.title('Динамика средней скорости по кластерам')
plt.xlabel('Год')
plt.ylabel('Средняя скорость')
plt.show()


In [ ]:
# Прогнозирование средней длины поездки для кластера 0
cluster_0 = data[data['cluster'] == 0].groupby('year')['road_length'].mean().reset_index()

# Подготовка данных
years = cluster_0['year'].values.reshape(-1, 1)
road_lengths = cluster_0['road_length'].values

# Линейная регрессия
model = LinearRegression()
model.fit(years, road_lengths)

# Прогноз на 2 года вперед
future_years = np.array([2023, 2024]).reshape(-1, 1)
predictions = model.predict(future_years)

# Визуализация прогноза
plt.plot(years, road_lengths, marker='o', label='Фактические данные')
plt.plot(future_years, predictions, marker='x', label='Прогноз')
plt.title('Прогноз средней длины поездки (Кластер 0)')
plt.xlabel('Год')
plt.ylabel('Средняя длина поездки')
plt.legend()
plt.show()

print("Прогноз на 2023 и 2024 годы:", predictions)


In [ ]:
from fastapi import FastAPI
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from pydantic import BaseModel

app = FastAPI()

# Пример данных
data = pd.DataFrame({
    "year": [2020, 2021, 2022],
    "road_length": [10.5, 12.3, 11.8],
    "travel_time": [1.2, 1.3, 1.1],
    "tips_amount": [2.5, 3.0, 2.8],
})

# Кластеризация
kmeans = KMeans(n_clusters=3, random_state=42)
data['cluster'] = kmeans.fit_predict(data[['road_length', 'travel_time', 'tips_amount']])

# Модель для прогноза
reg_models = {}
for cluster in data['cluster'].unique():
    cluster_data = data[data['cluster'] == cluster]
    model = LinearRegression()
    model.fit(cluster_data[['year']], cluster_data['road_length'])
    reg_models[cluster] = model


class PredictionRequest(BaseModel):
    cluster: int
    year: int


@app.get("/")
def read_root():
    return {"message": "Добро пожаловать в API для анализа перевозок!"}


@app.get("/clusters")
def get_clusters():
    """Возвращает список кластеров."""
    clusters_info = data.groupby('cluster').mean().reset_index().to_dict(orient="records")
    return {"clusters": clusters_info}


@app.post("/predict")
def predict(request: PredictionRequest):
    """Прогноз на основе линейной регрессии."""
    cluster = request.cluster
    year = request.year

    if cluster not in reg_models:
        return {"error": "Кластер не найден"}

    model = reg_models[cluster]
    prediction = model.predict([[year]])
    return {"year": year, "cluster": cluster, "predicted_road_length": prediction[0]}


In [ ]:
import streamlit as st
import requests
import pandas as pd
import matplotlib.pyplot as plt

API_URL = "http://127.0.0.1:8000"

# Заголовок приложения
st.title("Анализ и прогноз перевозок")

# Справка
st.sidebar.title("Справка")
st.sidebar.write("""
1. Выберите кластер для анализа.
2. Укажите год для прогноза.
3. Нажмите "Показать результаты" для получения данных.
""")

# Получение кластеров
response = requests.get(f"{API_URL}/clusters")
clusters = response.json()['clusters']

# Выбор кластера
st.sidebar.subheader("Выбор кластера")
cluster = st.sidebar.selectbox(
    "Выберите кластер:",
    [c['cluster'] for c in clusters]
)

# Выбор года
st.sidebar.subheader("Прогноз на год")
year = st.sidebar.number_input("Введите год (например, 2025):", min_value=2023, step=1)

# Кнопка выполнения
if st.sidebar.button("Показать результаты"):
    # Запрос к API
    prediction_response = requests.post(f"{API_URL}/predict", json={"cluster": cluster, "year": year})
    prediction = prediction_response.json()

    if "error" in prediction:
        st.error(prediction["error"])
    else:
        st.success(f"Прогноз для кластера {cluster} на {year} год:")
        st.write(f"Предсказанная длина дороги: {prediction['predicted_road_length']:.2f} км")

        # Визуализация
        cluster_data = pd.DataFrame(clusters)
        st.subheader("Динамика длины дороги по годам")
        plt.plot(cluster_data['year'], cluster_data['road_length'], marker='o', label=f'Кластер {cluster}')
        plt.axhline(y=prediction['predicted_road_length'], color='r', linestyle='--', label='Прогноз')
        plt.xlabel("Год")
        plt.ylabel("Длина дороги (км)")
        plt.legend()
        st.pyplot(plt)


In [ ]:
uvicorn app:app --reload


In [ ]:
streamlit run app_ui.py
